### Exploracion de datos faltantes de un restaurante

In [5]:
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot
from fs import open_fs
from pathlib import Path
import shutil # Necesario para la operación de movimiento de archivos
import sys
from pathlib import Path

# This code snippet is used to import a custom Python module named 'pandas_missing_extension.py'
# located in the parent directory of the current notebook.
# 1. Get the path of the current notebook (where live-exploration-missing-values.ipynb is)
#    This will give you something like:
#    /home/paco/datos_faltantes/curso-datos-faltantes-main/jupyter/live-exploration-missing-values.ipynb
current_notebook_path = Path.cwd() # Path.cwd() returns the current working directory of the kernel

# 2. Navigate up one level to find the directory containing 'pandas_missing_extension.py'
#    This will give you:
#    /home/paco/datos_faltantes/curso-datos-faltantes-main/
project_root_dir = current_notebook_path.parent

# 3. Add this directory to Python's system path if it's not already there
#    This allows Python to find modules in this directory.
if str(project_root_dir) not in sys.path:
    sys.path.append(str(project_root_dir))
    print(f"Added {project_root_dir} to sys.path")

# 4. Now, import your custom accessor module
#    Python will now find 'pandas_missing_extension.py' in the added path.
import pandas_missing_extension
import warnings

# Puedes ser más específico con el tipo de advertencia si lo conoces, por ejemplo:
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

# Para las advertencias relacionadas con pkg_resources (como las de tu captura):
warnings.filterwarnings("ignore", message="pkg_resources is deprecated as an API")
warnings.filterwarnings("ignore", message="Deprecated call to pkg_resources.declare_namespace")


_Carga de datos_

In [6]:
# Definir la ruta relativa directamente desde la ubicación del notebook
# '../' sube un nivel, 'data/' entra al directorio 'data'
csv_file_path = Path("../data/Datos_restaurante_con_valores_faltantes.csv")

# Verificar si la ruta del archivo existe (opcional, pero buena práctica)
if csv_file_path.exists():
    # Leer el archivo CSV en un DataFrame de pandas
    df = pd.read_csv(csv_file_path)
    print("Archivo CSV cargado exitosamente.")
    print(df.head()) # Muestra las primeras filas del DataFrame
else:
    print(f"Error: El archivo no se encontró en la ruta: {csv_file_path}")

Archivo CSV cargado exitosamente.
   ID_cliente  Edad Género Frecuencia_visita  Monto_gastado Tipo_pago  \
0         1.0  56.0      M           Semanal     929.686812       NaN   
1         2.0  69.0      M           Semanal     179.643252       NaN   
2         NaN  46.0      M           Semanal     276.384576   Tarjeta   
3         4.0  32.0      M            Diaria     140.704560       App   
4         5.0  60.0      F           Semanal     392.797298       App   

   Tiempo_espera  Calificación_servicio Plato_favorito Bebida_favorita  \
0           36.0                    3.0          Pasta         Cerveza   
1            8.0                    1.0          Pasta        Refresco   
2           34.0                    5.0            NaN             NaN   
3           41.0                    1.0          Tacos            Agua   
4           27.0                    3.0          Pasta         Cerveza   

  Recomendó Visitó_fin_de_semana    Propina  Satisfacción Comentario  \
0     Fals